In [34]:
import pandas as pd
import numpy as np

In [35]:
data = pd.read_csv('imdb5000_dataset.csv')

In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5043 entries, 0 to 5042
Data columns (total 28 columns):
color                        5024 non-null object
director_name                4939 non-null object
num_critic_for_reviews       4993 non-null float64
duration                     5028 non-null float64
director_facebook_likes      4939 non-null float64
actor_3_facebook_likes       5020 non-null float64
actor_2_name                 5030 non-null object
actor_1_facebook_likes       5036 non-null float64
gross                        4159 non-null float64
genres                       5043 non-null object
actor_1_name                 5036 non-null object
movie_title                  5043 non-null object
num_voted_users              5043 non-null int64
cast_total_facebook_likes    5043 non-null int64
actor_3_name                 5020 non-null object
facenumber_in_poster         5030 non-null float64
plot_keywords                4890 non-null object
movie_imdb_link              5043 non-

In [37]:
#Выбираем интересующие нас фичи и выкидываем наны
df = data[['duration', 'genres', 'imdb_score', 'title_year']].copy()
df.dropna(inplace = True)
df.reset_index(inplace = True)

In [38]:
genres = df['genres'].str.split('|', expand = True)
genre_list = pd.unique(genres[[0,1,2,3,4,5,6,7]].values.ravel('K'))
#Удаляем None из списка жанров
genre_list = np.delete(genre_list, 20)
films_genres = pd.DataFrame()
#Создаём "индикаторную" таблицу
d = pd.DataFrame(False, index=np.arange(len(df)), columns=genre_list)   

In [39]:
for row in genres.itertuples():
    for genre in genre_list:
        if genre in row:
            d.at[row[0], genre] = True     

In [50]:
df1 = pd.concat([df,d],axis=1)

,index,director_name,duration,genres,imdb_score,title_year,Action,Adventure,Drama,Animation,...,Family,Music,Western,Musical,Film-Noir,History,War,Sport,Short,News
0,0,James Cameron,178.0,Action|Adventure|Fantasy|Sci-Fi,7.9,2009.0,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,Gore Verbinski,169.0,Action|Adventure|Fantasy,7.1,2007.0,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2,Sam Mendes,148.0,Action|Adventure|Thriller,6.8,2015.0,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,3,Christopher Nolan,164.0,Action|Thriller,8.5,2012.0,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,5,Andrew Stanton,132.0,Action|Adventure|Sci-Fi,6.6,2012.0,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
5,6,Sam Raimi,156.0,Action|Adventure|Romance,6.2,2007.0,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
6,7,Nathan Greno,100.0,Adventure|Animation|Comedy|Family|Fantasy|Musi...,7.8,2010.0,False,True,False,True,...,True,False,False,True,False,False,False,False,False,False
7,8,Joss Whedon,141.0,Action|Adventure|Sci-Fi,7.5,2015.0,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
8,9,David Yates,153.0,Adventure|Family|Fantasy|Mystery,7.5,2009.0,False,True,False,False,...,True,False,False,False,False,False,False,False,False,False
9,10,Zack Snyder,183.0,Action|Adventure|Sci-Fi,6.9,2016.0,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [41]:
df1.drop('genres', axis = 1, inplace = True)
df1.dropna(inplace = True)

In [43]:
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.datasets.samples_generator import make_blobs

In [44]:
bandwidth = estimate_bandwidth(df1, quantile=0.2, n_samples=500)

ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
ms.fit(df1)
labels = ms.labels_
cluster_centers = ms.cluster_centers_

labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)

print("number of estimated clusters : %d" % n_clusters_)

number of estimated clusters : 6


In [45]:
s = pd.DataFrame(labels)
df1['cluster'] = s
#df1['cluster'].value_counts()
df1.head(20)

,index,duration,imdb_score,title_year,Action,Adventure,Drama,Animation,Comedy,Mystery,...,Music,Western,Musical,Film-Noir,History,War,Sport,Short,News,cluster
0,0,178.0,7.9,2009.0,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,3
1,1,169.0,7.1,2007.0,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,3
2,2,148.0,6.8,2015.0,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,3
3,3,164.0,8.5,2012.0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,3
4,5,132.0,6.6,2012.0,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,3
5,6,156.0,6.2,2007.0,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,3
6,7,100.0,7.8,2010.0,False,True,False,True,True,False,...,False,False,True,False,False,False,False,False,False,3
7,8,141.0,7.5,2015.0,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,3
8,9,153.0,7.5,2009.0,False,True,False,False,False,True,...,False,False,False,False,False,False,False,False,False,3
9,10,183.0,6.9,2016.0,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,3
